Note: this won't work on your system unless you download the datasets and order them in the same way I have

In [253]:
import pandas as pd
import numpy as np
import io
import os
import altair as alt

In [254]:
# Import datetime so we can treat date columns as such and perform operations
from datetime import datetime as dt

Imports of packages

In [255]:
#os.chdir("C:\\Users\\Alexa\\OneDrive\\Ottawa U\\Year 4\\Fall\\MAT4376 - Data Analysis\\Project 1")
# set this to wherever the provided dataset is located

In [256]:
os.getcwd()

'C:\\Users\\awyat049\\OneDrive\\Ottawa U\\Year 4\\Fall\\MAT4376 - Data Analysis\\Project 1\\MAT4376-P1'

In [257]:
as_data = pd.read_csv('polls_us_election_2016.csv')

In [258]:
whole_data = pd.read_csv('Dataset_assigned_data_taken_from/president_general_polls_2016.csv')

In [259]:
poll_raw_years = pd.read_csv('2016_Pollster_Ratings/raw-polls.csv')

In [260]:
poll_ratings_bias = pd.read_csv('2016_Pollster_Ratings/pollster-stats-full.csv')

In [261]:
pres_pop = pd.read_csv('2016_pres_pop_vote.csv', index_col=0).dropna()

***

In [262]:
# convert all dates into datetime format
as_data['startdate'] = as_data['startdate'].astype('datetime64[ns]')
as_data['enddate'] = as_data['enddate'].astype('datetime64[ns]')
whole_data['startdate'] = whole_data['startdate'].astype('datetime64[ns]')
whole_data['enddate'] = whole_data['enddate'].astype('datetime64[ns]')
# Note: forecast date in whole_data is WRONG - interpreted month as year
poll_raw_years['polldate'] = poll_raw_years['polldate'].astype('datetime64[ns]')
poll_raw_years['electiondate'] = poll_raw_years['electiondate'].astype('datetime64[ns]')

In [263]:
#only 2016 polls

poll_raw_years = poll_raw_years[poll_raw_years['year'] == 2016]
poll_raw_years['electiondate'].unique

<bound method Series.unique of 7634   2016-02-01
7635   2016-02-01
7636   2016-02-01
7637   2016-02-01
7638   2016-02-01
          ...    
9003   2016-11-08
9004   2016-12-10
9005   2016-12-10
9006   2016-12-10
9007   2016-12-10
Name: electiondate, Length: 1374, dtype: datetime64[ns]>

In [264]:
# match pols from 'whole_data' and 'poll_raw_years'
# first cut down 'poll_raw_years' to just 2016 and just November Elections

poll_raw_years = poll_raw_years[poll_raw_years['electiondate'] == np.datetime64('2016-11-08')]

#contributor_polls = poll_raw_years[poll_raw_years['pollster']=='Gravis Marketing/Kaplan Strategies']
#contributor_polls
poll_raw_years = poll_raw_years[poll_raw_years['type_simple'] == 'Pres-G']
#poll_raw_years.columns

***

In [265]:
# select all cells from original set that have a 'poll_id', 'question_id', 'samplesize'

final_matched_set = whole_data[whole_data['poll_id'].isin(poll_raw_years['poll_id'].unique())]
#final_matched_set

In [266]:
final_matched_set = final_matched_set[final_matched_set['question_id'].isin(poll_raw_years['question_id'].unique())]
#final_matched_set.head(10)

In [267]:
#final_matched_set.loc[8]
final_matched_set.shape[0]
final_matched_set = final_matched_set.reset_index()
#final_matched_set
#final_matched_set.iloc[8]

In [268]:
poll_raw_years_indices = [[].copy() for i in range(final_matched_set.shape[0])]
matching_pollsters = [[].copy() for i in range(final_matched_set.shape[0])]
matching_methodology = [0]*final_matched_set.shape[0]#[].copy() for i in range(final_matched_set.shape[0])]
matching_bias = [[].copy() for i in range(final_matched_set.shape[0])]

i = 0

# note: index is the LABELLED index
for index, row in final_matched_set.iterrows():
    
    poll_raw_years_indices[i] = poll_raw_years.index[poll_raw_years['poll_id'] == row['poll_id']].tolist()[0]
    
    matching_pollsters[i] = (poll_raw_years.loc[poll_raw_years_indices[i]])['pollster']
    matching_methodology[i] = (poll_raw_years.loc[poll_raw_years_indices[i]])['methodology']
    i += 1

#for i in poll_raw_years_indices:
    
final_matched_set['methodology'] = matching_methodology

print("done")
    
# now it's confirmed that all our rows are correct, so next within the same code APPEND THE PROPER CELL FOR METHODOLOGY 
## Will need to use the strategy of making a new list then appaending that list as a column

#need to find the index in poll_raw_years that corresponds to each poll_id, and then check if the the 'question_id' also corresponds, and the sample size
#but there will be multiple entries with each poll_id, question_id etc, so we need to check ALL with matching poll_id and
# keep only those that match all three of a given row in final_matched_set

# It might be easier to identify and append rows

done


***

In [269]:
#poll_raw_years.index[(poll_raw_years['poll_id']==47324)].tolist()
#len(poll_raw_years_indices)
#final_matched_set
unmatched_entries = []
indices = []

for index, row in final_matched_set.iterrows():
    #print(str(row['pollster']) + " : " + str(matching_pollsters[index]))
    if row['pollster'] != matching_pollsters[index]:
        if row['pollster'] not in (matching_pollsters[index]) and matching_pollsters[index] not in (row['pollster']):
            unmatched_entries.append(str(row['pollster']) + " : " + str(matching_pollsters[index]))
            indices.append(index)
        
unmatched = pd.DataFrame({'org_index':indices,
                        'pollsters':unmatched_entries})
#unmatched

In [270]:
#"Lucid" not in "The Times-Picayune/Lucid"
#unmatched.to_csv('out.csv',index=False)
final_test = final_matched_set.copy()

In [271]:
# of the matched polls this was the only pollster that could not be verified as being the same under both datasets
# we drop it for consistency
final_test = final_test.drop(final_test.loc[final_test['pollster'] == 'University of Denver'].index)
#final_test.columns
# this is the final set

In [272]:
assigned_compare = pd.read_csv('polls_us_election_2016.csv')
assigned_compare['startdate'] = assigned_compare['startdate'].astype('datetime64[ns]')
assigned_compare['enddate'] = assigned_compare['enddate'].astype('datetime64[ns]')

In [273]:
# let's see how many of these are in the original dataset
org_test = final_test.copy()

# take irrelevant columns out to allow testing for equality
unique_cols = assigned_compare.columns.append(org_test.columns).drop_duplicates(keep=False)

org_test = org_test.drop(columns=unique_cols)


# inner join and check size

end = org_test.merge(as_data, 'inner')
len(end['pollster'].unique())
len(final_test['pollster'].unique())
final_test.shape[0]
final_test['methodology'].unique()

array(['Online', 'IVR/Online', 'IVR', 'Live Phone', 'IVR/Live Phone',
       'Mail', 'Live Phone/Online', 'IVR/Live Phone/Text'], dtype=object)

***

# number of items depending on dataset

- Assigned dataset would have 418 entries
- Origin dataset would have 1254 entries

- of the 418 entries from assigned, there would be 88 unique pollsters (end)
- of the 1254 entries from the whole set, again, 88 are unique. (final_test)
-- go with BIGGER SET, some values are necessary that are not in assigned for identifying and more data points should make better results

## Rethink this if necessary

# bar chart, x axis is methodology, dual bar where one is percentage republican, other is percentage dem, 
# and y-axis is the actual percentages (adjusted and raw) - Note: adjusted takes into factors like pollster rating

US_chart = final_test[final_test['state'] == 'U.S.']

chart_US = alt.Chart(US_chart).mark_bar().encode(
    column = alt.Column('state'),
    x = alt.X('methodology'),
    y = alt.Y('mean(rawpoll_clinton)'),
    color = alt.Color('methodology')
).configure_facet(spacing = 0.3)

chart_all_raw = alt.Chart(final_test).mark_bar().encode(
    column = alt.Column('state'),
    x = alt.X('methodology'),
    y = alt.Y('mean(rawpoll_clinton)'),
    color = alt.Color('methodology')
).configure_facet(spacing = 0.3)

chart_all_adj = alt.Chart(final_test).mark_bar().encode(
    column = alt.Column('state'),
    x = alt.X('methodology'),
    y = alt.Y('mean(adjpoll_clinton)'),
    color = alt.Color('methodology')
).configure_facet(spacing = 0.3)

chart_all_adj.display()

### drop all of 'IVR/Live Phone/Text' and 'Mail'
### drop states where only one methodology
### States with only one:

- Alabama - Online
- Connecticut - Online
- Delaware - Online
- DC - Online
- Hawaii - Online
- Maryland - Online
- Mississippi - Online
- Montana - Online
- Nebraska - Online
- Nebraska CD-2 (second congressional district) - Live Phone
- North Dakota - Online
- Rhode Island - Online
- Tennessee - Online
- West Virginia - Online
- Wyoming - Online

## NOTE: IVR is over the phone so might want to combine with Live Phone

In [274]:
cut_down_data = final_test.copy()
cut_down_data = cut_down_data.drop(cut_down_data.loc[cut_down_data['methodology'] == 'IVR/Live Phone/Text'].index)
cut_down_data = cut_down_data.drop(cut_down_data.loc[cut_down_data['methodology'] == 'Mail'].index)
cut_down_data = cut_down_data.drop(cut_down_data.loc[cut_down_data['state'] == 'Alabama'].index)
cut_down_data = cut_down_data.drop(cut_down_data.loc[cut_down_data['state'] == 'Connecticut'].index)
cut_down_data = cut_down_data.drop(cut_down_data.loc[cut_down_data['state'] == 'Delaware'].index)
cut_down_data = cut_down_data.drop(cut_down_data.loc[cut_down_data['state'] == 'District of Columbia'].index)
cut_down_data = cut_down_data.drop(cut_down_data.loc[cut_down_data['state'] == 'Hawaii'].index)
cut_down_data = cut_down_data.drop(cut_down_data.loc[cut_down_data['state'] == 'Maryland'].index)
cut_down_data = cut_down_data.drop(cut_down_data.loc[cut_down_data['state'] == 'Mississippi'].index)
cut_down_data = cut_down_data.drop(cut_down_data.loc[cut_down_data['state'] == 'Montana'].index)
cut_down_data = cut_down_data.drop(cut_down_data.loc[cut_down_data['state'] == 'Nebraska'].index)
cut_down_data = cut_down_data.drop(cut_down_data.loc[cut_down_data['state'] == 'Nebraska CD-2'].index)
cut_down_data = cut_down_data.drop(cut_down_data.loc[cut_down_data['state'] == 'North Dakota'].index)
cut_down_data = cut_down_data.drop(cut_down_data.loc[cut_down_data['state'] == 'Rhode Island'].index)
cut_down_data = cut_down_data.drop(cut_down_data.loc[cut_down_data['state'] == 'Tennessee'].index)
cut_down_data = cut_down_data.drop(cut_down_data.loc[cut_down_data['state'] == 'West Virginia'].index)
cut_down_data = cut_down_data.drop(cut_down_data.loc[cut_down_data['state'] == 'Wyoming'].index)

#cut_down_data

In [275]:
# Build a set of ONLY the swing states
cut_down_swing = cut_down_data.copy()

cut_down_swing = cut_down_swing[cut_down_swing['state'] == 'Arizona']
cut_down_swing = pd.concat([cut_down_swing,cut_down_data[cut_down_data['state'] == 'Colorado']])
cut_down_swing = pd.concat([cut_down_swing,cut_down_data[cut_down_data['state'] == 'Florida']])
cut_down_swing = pd.concat([cut_down_swing,cut_down_data[cut_down_data['state'] == 'Iowa']])
cut_down_swing = pd.concat([cut_down_swing,cut_down_data[cut_down_data['state'] == 'Michigan']])
cut_down_swing = pd.concat([cut_down_swing,cut_down_data[cut_down_data['state'] == 'Nevada']])
cut_down_swing = pd.concat([cut_down_swing,cut_down_data[cut_down_data['state'] == 'New Hampshire']])
cut_down_swing = pd.concat([cut_down_swing,cut_down_data[cut_down_data['state'] == 'North Carolina']])
cut_down_swing = pd.concat([cut_down_swing,cut_down_data[cut_down_data['state'] == 'Ohio']])
cut_down_swing = pd.concat([cut_down_swing,cut_down_data[cut_down_data['state'] == 'Pennsylvania']])
cut_down_swing = pd.concat([cut_down_swing,cut_down_data[cut_down_data['state'] == 'Virginia']])
cut_down_swing = pd.concat([cut_down_swing,cut_down_data[cut_down_data['state'] == 'Wisconsin']])
cut_down_swing = pd.concat([cut_down_swing,cut_down_data[cut_down_data['state'] == 'U.S.']])


cut_down_swing

index  cycle     branch        type                        matchup  \
30       42   2016  President  polls-plus  Clinton vs. Trump vs. Johnson   
60       76   2016  President  polls-plus  Clinton vs. Trump vs. Johnson   
75       94   2016  President  polls-plus  Clinton vs. Trump vs. Johnson   
107     135   2016  President  polls-plus  Clinton vs. Trump vs. Johnson   
126     157   2016  President  polls-plus  Clinton vs. Trump vs. Johnson   
...     ...    ...        ...         ...                            ...   
1232  10412   2016  President  polls-only  Clinton vs. Trump vs. Johnson   
1234  10428   2016  President  polls-only  Clinton vs. Trump vs. Johnson   
1243  10542   2016  President  polls-only  Clinton vs. Trump vs. Johnson   
1255  10800   2016  President  polls-only  Clinton vs. Trump vs. Johnson   
1256  11071   2016  President  polls-only  Clinton vs. Trump vs. Johnson   

     forecastdate    state  startdate    enddate                  pollster  \
30        11/8/16  Arizona 2016-11-03 2016-11-06          Gravis Marketing   
60        11/8/16  Arizona 2016-10-29 2016-10-31        Saguaro Strategies   
75        11/8/16  Arizona 2016-11-01 2016-11-07              SurveyMonkey   
107       11/8/16  Arizona 2016-10-26 2016-10-28                    YouGov   
126       11/8/16  Arizona 2016-10-28 2016-11-01  The Times-Picayune/Lucid   
...           ...      ...        ...        ...                       ...   
1232      11/8/16     U.S. 2016-10-18 2016-10-24     USC Dornsife/LA Times   
1234      11/8/16     U.S. 2016-10-20 2016-10-23          Gravis Marketing   
1243      11/8/16     U.S. 2016-10-21 2016-10-23  The Times-Picayune/Lucid   
1255      11/8/16     U.S. 2016-10-18 2016-10-20  The Times-Picayune/Lucid   
1256      11/8/16     U.S. 2016-10-17 2016-10-23              SurveyMonkey   

      ... adjpoll_trump  adjpoll_johnson adjpoll_mcmullin  multiversions  \
30    ...      44.33273         3.726098              NaN            NaN   
60    ...      44.02861         6.357814              NaN            NaN   
75    ...      41.32993         4.692430              NaN            NaN   
107   ...      45.29469         4.397894              NaN            NaN   
126   ...      44.98566         6.366840              NaN            NaN   
...   ...           ...              ...              ...            ...   
1232  ...      42.30230              NaN              NaN            NaN   
1234  ...      50.19434              NaN              NaN            NaN   
1243  ...      42.57230         4.420021              NaN            NaN   
1255  ...      41.99739         3.061498              NaN            NaN   
1256  ...      42.97135         3.088467              NaN            NaN   

                                                    url  poll_id  question_id  \
30    http://www.gravispolls.com/2016/11/multi-state...    48851        76447   
60    https://www.scribd.com/document/329707080/Sagu...    48185        75456   
75    https://www.surveymonkey.com/elections/map?pol...    48862        76552   
107   http://www.cbsnews.com/news/cbs-battleground-p...    47778        74788   
126   https://luc.id/wp-content/uploads/2016/11/Ariz...    48102        75313   
...                                                 ...      ...          ...   
1232                       http://latimes.com/politics/    47320        74038   
1234  http://www.realclearpolitics.com/docs/2016/Gra...    47344        74085   
1243          https://luc.id/2016-presidential-tracker/    47258        73958   
1255          https://luc.id/2016-presidential-tracker/    47163        73806   
1256  https://www.scribd.com/document/328753246/NBC-...    47324        74045   

      createddate             timestamp  methodology  
30        11/7/16  09:35:33  8 Nov 2016   IVR/Online  
60        11/3/16  09:35:33  8 Nov 2016       Online  
75        11/8/16  09:35:33  8 Nov 2016       Online  
107      10/30/16  09:35:33  8 Nov 2016       Onli

In [276]:
swings = cut_down_swing['state'].unique()
pres_pop_pct_clinton = [0]*cut_down_swing.shape[0]
pres_pop_pct_trump = [0]*cut_down_swing.shape[0]
i = 0

for index,row in cut_down_swing.iterrows():
    
    pres_pop_pct_clinton[i] = pres_pop.loc[row['state'],'Clinton_Pct']
    pres_pop_pct_trump[i] = pres_pop.loc[row['state'],'Trump_Pct']
    i += 1
 


In [277]:
cut_down_swing['Clinton_Pct_Elec'] = pres_pop_pct_clinton
cut_down_swing['Trump_Pct_Elec'] = pres_pop_pct_trump
#cut_down_swing

chart_all_cut_adj = alt.Chart(cut_down_data).mark_bar().encode(
    column = alt.Column('state'),
    x = alt.X('methodology'),
    y = alt.Y('mean(adjpoll_clinton)'),
    color = alt.Color('methodology')
).configure_facet(spacing = 0.3)

chart_all_cut_adj

chart_all_cut_raw = alt.Chart(cut_down_data).mark_bar().encode(
    column = alt.Column('state'),
    x = alt.X('methodology'),
    y = alt.Y('mean(rawpoll_clinton)'),
    color = alt.Color('methodology')
).configure_facet(spacing = 0.3)

chart_all_cut_raw

In [278]:
# Only IVR, Live Phone, Online
ivr_phone_online = cut_down_data.copy()

ivr_phone_online = ivr_phone_online.drop(ivr_phone_online.loc[cut_down_data['methodology'] == 'IVR/Live Phone'].index)
ivr_phone_online = ivr_phone_online.drop(ivr_phone_online.loc[cut_down_data['methodology'] == 'IVR/Online'].index)
ivr_phone_online = ivr_phone_online.drop(ivr_phone_online.loc[cut_down_data['methodology'] == 'Live Phone/Online'].index)

#ivr_phone_online

ivr_phone_online_chart_raw = alt.Chart(ivr_phone_online).mark_bar().encode(
    column = alt.Column('state'),
    x = alt.X('methodology'),
    y = alt.Y('mean(rawpoll_clinton)'),
    color = alt.Color('methodology')
).configure_facet(spacing = 5)

ivr_phone_online_chart_raw

ivr_phone_online_chart_adj = alt.Chart(ivr_phone_online).mark_bar().encode(
    column = alt.Column('state'),
    x = alt.X('methodology'),
    y = alt.Y('mean(adjpoll_clinton)'),
    color = alt.Color('methodology')
).configure_facet(spacing = 5)

ivr_phone_online_chart_adj

In [279]:
ivr_phone_online_swing = cut_down_swing.copy()

#ivr_phone_online_swing = ivr_phone_online_swing.drop(ivr_phone_online_swing.loc[cut_down_swing['methodology'] == 'IVR/Live Phone'].index)
ivr_phone_online_swing = ivr_phone_online_swing.drop(ivr_phone_online_swing.loc[cut_down_swing['methodology'] == 'IVR/Online'].index)
ivr_phone_online_swing = ivr_phone_online_swing.drop(ivr_phone_online_swing.loc[cut_down_swing['methodology'] == 'Live Phone/Online'].index)

#ivr_phone_online_swing


ivr_phone_online_chart_swing_adj = alt.Chart(ivr_phone_online_swing).mark_bar().encode(
    column = alt.Column('state'),
    x = alt.X('methodology'),
    y = alt.Y('mean(adjpoll_clinton)'),
    color = alt.Color('methodology')
).configure_facet(spacing = 5)

ivr_phone_online_chart_swing_adj

ivr_phone_online_chart_swing_raw = alt.Chart(ivr_phone_online_swing).mark_bar().encode(
    column = alt.Column('state'),
    x = alt.X('methodology'),
    y = alt.Y('mean(rawpoll_clinton)'),
    color = alt.Color('methodology')
).configure_facet(spacing = 5)

ivr_phone_online_chart_swing_raw

three_trump_chart_swing_raw = alt.Chart(ivr_phone_online_swing).mark_bar().encode(
    column = alt.Column('state'),
    x = alt.X('methodology'),
    y = alt.Y('mean(rawpoll_trump)'),
    color = alt.Color('methodology')
).configure_facet(spacing = 5)

three_trump_chart_swing_raw

three_trump_chart_swing_adj = alt.Chart(ivr_phone_online_swing).mark_bar().encode(
    column = alt.Column('state'),
    x = alt.X('methodology'),
    y = alt.Y('mean(adjpoll_trump)'),
    color = alt.Color('methodology')
).configure_facet(spacing = 5)

three_trump_chart_swing_adj

In [280]:
# Combine IVR and Live Phone -> into Phone Based

phone_online_swing = ivr_phone_online_swing.copy()

phone_online_swing.loc[phone_online_swing['methodology'] == 'IVR','methodology'] = 'Phone'
phone_online_swing.loc[phone_online_swing['methodology'] == 'Live Phone','methodology'] = 'Phone'
phone_online_swing.loc[phone_online_swing['methodology'] == 'IVR/Live Phone','methodology'] = 'Phone'

#phone_online_swing['methodology']

In [281]:
# Remove all nan grades

phone_online_swing = phone_online_swing[phone_online_swing['grade'].notna()]
phone_online_swing['grade'].unique()

array(['C-', 'B', 'A+', 'C', 'B+', 'C+', 'B-', 'A-', 'A'], dtype=object)

phone_online_chart_swing_raw_clint = alt.Chart(phone_online_swing).mark_bar().encode(
    column = alt.Column('state'),
    x = alt.X('methodology'),
    y = alt.Y('mean(rawpoll_clinton)'),
    color = alt.Color('methodology')
)#.configure_facet(spacing = 5)

phone_online_chart_swing_raw_clint

phone_online_chart_swing_raw_trump = alt.Chart(phone_online_swing).mark_bar().encode(
    column = alt.Column('state'),
    x = alt.X('methodology'),
    y = alt.Y('mean(rawpoll_trump)'),
    color = alt.Color('methodology')
)#.configure_facet(spacing = 5)

phone_online_chart_swing_raw_trump

phone_online_chart_swing_adj_clint = alt.Chart(phone_online_swing).mark_bar().encode(
    column = alt.Column('state'),
    x = alt.X('methodology'),
    y = alt.Y('mean(adjpoll_clinton)'),
    color = alt.Color('methodology')
)#.configure_facet(spacing = 5)

phone_online_chart_swing_adj_clint

phone_online_chart_swing_adj_trump = alt.Chart(phone_online_swing).mark_bar().encode(
    column = alt.Column('state'),
    x = alt.X('methodology'),
    y = alt.Y('mean(adjpoll_trump)'),
    color = alt.Color('methodology')
)#.configure_facet(spacing = 5)

phone_online_chart_swing_adj_trump

In [282]:
## BELOW HERE RESULTS FOR ONLY KEY TRUMP SWING STATES
phone_online_swing_all = phone_online_swing.copy()

# Drop states with no 

phone_online_swing_all['grade'].unique()

array(['C-', 'B', 'A+', 'C', 'B+', 'C+', 'B-', 'A-', 'A'], dtype=object)

In [283]:
# remove states with missing data

phone_online_swing_all = phone_online_swing_all.drop(phone_online_swing_all.loc[phone_online_swing_all['state'] == 'Iowa'].index)
phone_online_swing_all = phone_online_swing_all.drop(phone_online_swing_all.loc[phone_online_swing_all['state'] == 'Michigan'].index)
phone_online_swing_all = phone_online_swing_all.drop(phone_online_swing_all.loc[phone_online_swing_all['state'] == 'Nevada'].index)
phone_online_swing_all = phone_online_swing_all.drop(phone_online_swing_all.loc[phone_online_swing_all['state'] == 'Virginia'].index)
phone_online_swing_all = phone_online_swing_all.drop(phone_online_swing_all.loc[phone_online_swing_all['state'] == 'Wisconsin'].index)


In [284]:
# Drop Colorado and New Hampshire - comment out if want them######################
phone_online_swing_all = phone_online_swing_all.drop(phone_online_swing_all.loc[phone_online_swing_all['state'] == 'Colorado'].index)
phone_online_swing_all = phone_online_swing_all.drop(phone_online_swing_all.loc[phone_online_swing_all['state'] == 'New Hampshire'].index)

# DROP U.S. - COMMENT BACK IF WANT IT ###################
phone_online_swing_all = phone_online_swing_all.drop(phone_online_swing_all.loc[phone_online_swing_all['state'] == 'U.S.'].index)


In [285]:
# create dataframe with RESULTS OF ONLY GOOD POLLS (B graded and above)
# Remove all grades under a B:  {B-,C+,C, C-} 
phone_online_swing_good = phone_online_swing_all[~phone_online_swing_all['grade'].isin(['B-','C+','C', 'C-'])]

#phone_online_swing_good

***
# IGNORE THESE CHARTS FOR NOW

phone_online_swing_all_raw_clint = alt.Chart(phone_online_swing_all).mark_bar().encode(
    column = alt.Column('state'),
    x = alt.X('methodology'),
    y = alt.Y('mean(rawpoll_clinton)'),
    color = alt.Color('methodology')
)#.configure_facet(spacing = 5)

phone_online_swing_all_raw_clint

phone_online_swing_all_raw_trump = alt.Chart(phone_online_swing_all).mark_bar().encode(
    column = alt.Column('state'),
    x = alt.X('methodology'),
    y = alt.Y('mean(rawpoll_trump)'),
    color = alt.Color('methodology')
)#.configure_facet(spacing = 5)

phone_online_swing_all_raw_trump

phone_online_swing_all_adj_clint = alt.Chart(phone_online_swing_all).mark_bar().encode(
    column = alt.Column('state'),
    x = alt.X('methodology'),
    y = alt.Y('mean(adjpoll_clinton)'),
    color = alt.Color('methodology')
)#.configure_facet(spacing = 5)

phone_online_swing_all_adj_clint

phone_online_swing_all_adj_trump = alt.Chart(phone_online_swing_all).mark_bar().encode(
    column = alt.Column('state'),
    x = alt.X('methodology'),
    y = alt.Y('mean(adjpoll_trump)'),
    color = alt.Color('methodology')
)#.configure_facet(spacing = 5)

phone_online_swing_all_adj_trump

test1 = phone_online_swing_all[phone_online_swing_all['methodology'] == 'Phone']
test1['adjpoll_trump'].mean()

test2 = phone_online_swing_all[phone_online_swing_all['methodology'] == 'Online']
test2['adjpoll_trump'].mean()

test1['adjpoll_clinton'].mean()

test2['adjpoll_clinton'].mean()

# phone
test1['rawpoll_trump'].mean()
# online
test2['rawpoll_trump'].mean()
# phone
test1['rawpoll_clinton'].mean()
# online
test2['rawpoll_clinton'].mean()

# IGNORE CELLS ABOVE FOR NOW
***
## Creation of sets with all pollster graded polls (other than not graded)

In [286]:
## Make new set with mean of each raw set for each relevant state as column entry and a third column for final results

phone_data = phone_online_swing_all[phone_online_swing_all['methodology'] == 'Phone']
online_data = phone_online_swing_all[phone_online_swing_all['methodology'] == 'Online']

list_of_states = phone_online_swing_all['state'].unique().tolist()
trump_phone_mean = [0]*len(list_of_states)
trump_online_mean = [0]*len(list_of_states)
trump_final_percent = [0]*len(list_of_states)

clinton_phone_mean = [0]*len(list_of_states)
clinton_online_mean = [0]*len(list_of_states)
clinton_final_percent = [0]*len(list_of_states)

for i in range(len(list_of_states)):
    curr_set_phone = phone_data[phone_data['state'] == list_of_states[i]]
    curr_set_online = online_data[online_data['state'] == list_of_states[i]]
    
    trump_phone_mean[i] = curr_set_phone['rawpoll_trump'].mean()
    trump_online_mean[i] = curr_set_online['rawpoll_trump'].mean()
    clinton_phone_mean[i] = curr_set_phone['rawpoll_clinton'].mean()
    clinton_online_mean[i] = curr_set_online['rawpoll_clinton'].mean()
    trump_final_percent[i] = phone_online_swing_all[phone_online_swing_all['state'] == list_of_states[i]]['Trump_Pct_Elec'].mean()
    clinton_final_percent[i] = phone_online_swing_all[phone_online_swing_all['state'] == list_of_states[i]]['Clinton_Pct_Elec'].mean()


## with colorado and new hampshire

fin_df_all = pd.DataFrame({'state':list_of_states,
                       'trump_phone_pct':trump_phone_mean,
                      'clinton_phone_pct':clinton_phone_mean,
                      'trump_online_pct':trump_online_mean,
                      'clinton_online_pct':clinton_online_mean,
                      'trump_elect_pct':trump_final_percent,
                      'clinton_elect_pct':clinton_final_percent,
                      'won_by_pop': ['Donald Trump', 'Hillary Clinton', 'Donald Trump', 'Hillary Clinton', 
                                 'Donald Trump', 'Donald Trump', 'Donald Trump', 'Hillary Clinton'],
                      'won_by_seats':['Donald Trump', 'Hillary Clinton', 'Donald Trump', 'Hillary Clinton', 
                                 'Donald Trump', 'Donald Trump', 'Donald Trump', 'Donald Trump'],
                      'num_seats_winner': [11,9,29,4,15,18,20,304]})

fin_df_all

In [287]:
# Without Colorado and New Hampshire
## with colorado and new hampshire

fin_df_all = pd.DataFrame({'state':list_of_states,
                       'Phone Polls (Trump)':trump_phone_mean,
                      'Phone Polls (Clinton)':clinton_phone_mean,
                      'Online Polls (Trump)':trump_online_mean,
                      'Online Polls (Clinton)':clinton_online_mean,
                      'Election Vote (Trump)':trump_final_percent,
                      'Election Vote (Clinton)':clinton_final_percent,
                      'won_by_pop': ['Donald Trump', 'Donald Trump', 
                                 'Donald Trump', 'Donald Trump', 'Donald Trump'],# remember to add in final column if add U.S. back - TO DO
                      'won_by_seats':['Donald Trump', 'Donald Trump',
                                 'Donald Trump', 'Donald Trump', 'Donald Trump'],# remember to add in final column if add U.S. back - TO DO
                      'num_seats_winner': [11,29,15,18,20]})# remember to add in final column if add U.S. back - TO DO

#fin_df_all

***
## Creation of sets with only pollsters graded B or above

In [288]:
# Final Dataset and Visualizations (?) - before making pretty


# I want to insert a column on each graph with the actual result -- how to do
## Make new set with mean of each raw set for each relevant state as column entry and a third column for final results

phone_data_good = phone_online_swing_good[phone_online_swing_good['methodology'] == 'Phone']
online_data_good = phone_online_swing_good[phone_online_swing_good['methodology'] == 'Online']

list_of_states = phone_online_swing_good['state'].unique().tolist()
trump_phone_mean_good = [0]*len(list_of_states)
trump_online_mean_good = [0]*len(list_of_states)
trump_final_percent = [0]*len(list_of_states)

clinton_phone_mean_good = [0]*len(list_of_states)
clinton_online_mean_good = [0]*len(list_of_states)
clinton_final_percent = [0]*len(list_of_states)

for i in range(len(list_of_states)):
    curr_set_phone = phone_data_good[phone_data_good['state'] == list_of_states[i]]
    curr_set_online = online_data_good[online_data_good['state'] == list_of_states[i]]
    
    trump_phone_mean_good[i] = curr_set_phone['rawpoll_trump'].mean()
    trump_online_mean_good[i] = curr_set_online['rawpoll_trump'].mean()
    clinton_phone_mean_good[i] = curr_set_phone['rawpoll_clinton'].mean()
    clinton_online_mean_good[i] = curr_set_online['rawpoll_clinton'].mean()
    trump_final_percent[i] = phone_online_swing_good[phone_online_swing_good['state'] == list_of_states[i]]['Trump_Pct_Elec'].mean()
    clinton_final_percent[i] = phone_online_swing_good[phone_online_swing_good['state'] == list_of_states[i]]['Clinton_Pct_Elec'].mean()


In [465]:
# Without Colorado and New Hampshire
## with colorado and new hampshire

fin_df_good = pd.DataFrame({'state':list_of_states,
                       'Phone Polls (Trump)':trump_phone_mean_good,
                      'Phone Polls (Clinton)':clinton_phone_mean_good,
                      'Online Polls (Trump)':trump_online_mean_good,
                      'Online Polls (Clinton)':clinton_online_mean_good,
                      'Election Vote (Trump)':trump_final_percent,
                      'Election Vote (Clinton)':clinton_final_percent,
                      'won_by_pop': ['Donald Trump', 'Donald Trump', 
                                 'Donald Trump', 'Donald Trump', 'Donald Trump'],# remember to add in final column if add U.S. back - TO DO
                      'won_by_seats':['Donald Trump', 'Donald Trump',
                                 'Donald Trump', 'Donald Trump', 'Donald Trump'],# remember to add in final column if add U.S. back - TO DO
                      'num_seats_winner': [11,29,15,18,20]}) # remember to add in final column if add U.S. back - TO DO

fin_df_good

state  Phone Polls (Trump)  Phone Polls (Clinton)  \
0         Arizona            46.500000              44.000000   
1         Florida            42.000000              44.500000   
2  North Carolina            44.050000              45.250000   
3            Ohio            45.200000              42.966667   
4    Pennsylvania            41.666667              46.166667   

   Online Polls (Trump)  Online Polls (Clinton)  Election Vote (Trump)  \
0                  44.0                    42.0              48.671616   
1                  44.0                    45.5              49.021941   
2                  45.0                    48.0              49.828095   
3                  46.0                    45.0              51.687651   
4                  41.5                    46.5              48.183336   

   Election Vote (Clinton)    won_by_pop  won_by_seats  num_seats_winner  
0                45.126022  Donald Trump  Donald Trump                11  
1                47.823316  Donald Trump  Donald Trump                29  
2                46.172866  Donald Trump  Donald Trump                15  
3                43.558076  Donald Trump  Donald Trump                18  
4                47.464949  Donald Trump  Donald Trump                20

In [466]:
fin_df_all

state  Phone Polls (Trump)  Phone Polls (Clinton)  \
0         Arizona            46.500000              44.000000   
1         Florida            45.050000              45.318750   
2  North Carolina            45.388333              45.693333   
3            Ohio            46.035000              43.262500   
4    Pennsylvania            43.143000              46.151000   

   Online Polls (Trump)  Online Polls (Clinton)  Election Vote (Trump)  \
0             43.333333               43.333333              48.671616   
1             44.500000               46.000000              49.021941   
2             43.333333               47.666667              49.828095   
3             45.333333               42.333333              51.687651   
4             41.750000               47.000000              48.183336   

   Election Vote (Clinton)    won_by_pop  won_by_seats  num_seats_winner  
0                45.126022  Donald Trump  Donald Trump                11  
1                47.823316  Donald Trump  Donald Trump                29  
2                46.172866  Donald Trump  Donald Trump                15  
3                43.558076  Donald Trump  Donald Trump                18  
4                47.464949  Donald Trump  Donald Trump                20

In [290]:
fin_df_all_100 = fin_df_all.copy()


scaler_phone = 100/(fin_df_all_100['Phone Polls (Trump)'] + fin_df_all_100['Phone Polls (Clinton)'])
scaler_online = 100/(fin_df_all_100['Online Polls (Trump)'] + fin_df_all_100['Online Polls (Clinton)'])
scaler_elect = 100/(fin_df_all_100['Election Vote (Trump)'] + fin_df_all_100['Election Vote (Clinton)'])


fin_df_all_100['Phone Polls (Trump)'] = fin_df_all_100['Phone Polls (Trump)'] * scaler_phone
fin_df_all_100['Phone Polls (Clinton)'] = fin_df_all_100['Phone Polls (Clinton)'] * scaler_phone

fin_df_all_100['Online Polls (Trump)'] = fin_df_all_100['Online Polls (Trump)'] * scaler_online
fin_df_all_100['Online Polls (Clinton)'] = fin_df_all_100['Online Polls (Clinton)'] * scaler_online

fin_df_all_100['Election Vote (Trump)'] = fin_df_all_100['Election Vote (Trump)'] * scaler_elect
fin_df_all_100['Election Vote (Clinton)'] = fin_df_all_100['Election Vote (Clinton)'] * scaler_elect



In [291]:
fin_df_all_100

state  Phone Polls (Trump)  Phone Polls (Clinton)  \
0         Arizona            51.381215              48.618785   
1         Florida            49.851304              50.148696   
2  North Carolina            49.832568              50.167432   
3            Ohio            51.552395              48.447605   
4    Pennsylvania            48.315676              51.684324   

   Online Polls (Trump)  Online Polls (Clinton)  Election Vote (Trump)  \
0             50.000000               50.000000              51.890023   
1             49.171271               50.828729              50.618835   
2             47.619048               52.380952              51.903746   
3             51.711027               48.288973              54.267685   
4             47.042254               52.957746              50.375536   

   Election Vote (Clinton)    won_by_pop  won_by_seats  num_seats_winner  
0                48.109977  Donald Trump  Donald Trump                11  
1                49.381165  Donald Trump  Donald Trump                29  
2                48.096254  Donald Trump  Donald Trump                15  
3                45.732315  Donald Trump  Donald Trump                18  
4                49.624464  Donald Trump  Donald Trump                20

In [373]:
# Without Colorado and New Hampshire
## MAKE ONE FOR EACH STATE / U.S.

domain_1 = ['Grade B and Above (Trump)', 'All Polls (Trump)']
range_1 = ['#2d3142','#9a031e'] # make clinton the opposite? red sticking out blue on bottom? (so looks like blue fill)

fin_df_Arizona_phone = pd.DataFrame({'label':['Phone'],
                               'All Polls (Trump)':[trump_phone_mean[0]],
                               'All Polls (Clinton)':[clinton_phone_mean[0]],
                               'Grade B and Above (Trump)':[trump_phone_mean_good[0]],
                               'Grade B and Above (Clinton)':[clinton_phone_mean_good[0]]})

fin_df_Arizona_online = pd.DataFrame({'label':['Online'],
                               'All Polls (Trump)':[trump_online_mean[0]],
                               'All Polls (Clinton)':[clinton_online_mean[0]],
                               'Grade B and Above (Trump)':[trump_online_mean_good[0]],
                               'Grade B and Above (Clinton)':[clinton_online_mean_good[0]]})



fin_df_Florida_phone = pd.DataFrame({'label':['Phone'],
                               'All Polls (Trump)':[trump_phone_mean[1]],
                               'All Polls (Clinton)':[clinton_phone_mean[1]],
                               'Grade B and Above (Trump)':[trump_phone_mean_good[1]],
                               'Grade B and Above (Clinton)':[clinton_phone_mean_good[1]]})

fin_df_Florida_online = pd.DataFrame({'label':['Online'],
                               'All Polls (Trump)':[trump_online_mean[1]],
                               'All Polls (Clinton)':[clinton_online_mean[1]],
                               'Grade B and Above (Trump)':[trump_online_mean_good[1]],
                               'Grade B and Above (Clinton)':[clinton_online_mean_good[1]]})



fin_df_North_Carolina_phone = pd.DataFrame({'label':['Phone'],
                               'All Polls (Trump)':[trump_phone_mean[2]],
                               'All Polls (Clinton)':[clinton_phone_mean[2]],
                               'Grade B and Above (Trump)':[trump_phone_mean_good[2]],
                               'Grade B and Above (Clinton)':[clinton_phone_mean_good[2]]})

fin_df_North_Carolina_online = pd.DataFrame({'label':['Online'],
                               'All Polls (Trump)':[trump_online_mean[2]],
                               'All Polls (Clinton)':[clinton_online_mean[2]],
                               'Grade B and Above (Trump)':[trump_online_mean_good[2]],
                               'Grade B and Above (Clinton)':[clinton_online_mean_good[2]]})



fin_df_Ohio_phone = pd.DataFrame({'label':['Phone'],
                               'All Polls (Trump)':[trump_phone_mean[3]],
                               'All Polls (Clinton)':[clinton_phone_mean[3]],
                               'Grade B and Above (Trump)':[trump_phone_mean_good[3]],
                               'Grade B and Above (Clinton)':[clinton_phone_mean_good[3]]})

fin_df_Ohio_online = pd.DataFrame({'label':['Online'],
                               'All Polls (Trump)':[trump_online_mean[3]],
                               'All Polls (Clinton)':[clinton_online_mean[3]],
                               'Grade B and Above (Trump)':[trump_online_mean_good[3]],
                               'Grade B and Above (Clinton)':[clinton_online_mean_good[3]]})



fin_df_Pennsylvania_phone = pd.DataFrame({'label':['Phone'],
                               'All Polls (Trump)':[trump_phone_mean[4]],
                               'All Polls (Clinton)':[clinton_phone_mean[4]],
                               'Grade B and Above (Trump)':[trump_phone_mean_good[4]],
                               'Grade B and Above (Clinton)':[clinton_phone_mean_good[4]]})

fin_df_Pennsylvania_online = pd.DataFrame({'label':['Online'],
                               'All Polls (Trump)':[trump_online_mean[4]],
                               'All Polls (Clinton)':[clinton_online_mean[4]],
                               'Grade B and Above (Trump)':[trump_online_mean_good[4]],
                               'Grade B and Above (Clinton)':[clinton_online_mean_good[4]]})

#fin_df_Arizona

# old charts faceted

layered_Arizona = alt.Chart(fin_df_Arizona).transform_fold(
    ['trump_good','trump_all'], as_=['Grades','Percentage']).mark_bar(opacity=1).encode(
    x = alt.X('label:N'),
    y = alt.Y('Percentage:Q', scale = alt.Scale(domain = [35,52]), stack=None),
    color = alt.Color('Grades:N',scale = alt.Scale(domain=domain_1, range=range_1))).properties(width=200)

layered_Florida = alt.Chart(fin_df_Florida).transform_fold(
    ['trump_all','trump_good'], as_=['Grades','Percentage']).mark_bar(opacity=1).encode(
    x = alt.X('label:N'),
    y = alt.Y('Percentage:Q', scale = alt.Scale(domain = [35,52]), stack=None),
    color = alt.Color('Grades:N',scale = alt.Scale(domain=domain_1, range=range_1))).properties(width=200)

layered_North_Carolina = alt.Chart(fin_df_North_Carolina).transform_fold(
    ['trump_all','trump_good'], as_=['Grades','Percentage']).mark_bar(opacity=1).encode(
    x = alt.X('label:N'),
    y = alt.Y('Percentage:Q', scale = alt.Scale(domain = [35,52]), stack=None),
    color = alt.Color('Grades:N',scale = alt.Scale(domain=domain_1, range=range_1))).properties(width=200)

layered_Pennsylvania = alt.Chart(fin_df_Pennsylvania).transform_fold(
    ['trump_all','trump_good'], as_=['Grades','Percentage']).mark_bar(opacity=1).encode(
    x = alt.X('label:N'),
    y = alt.Y('Percentage:Q', scale = alt.Scale(domain = [35,52]), stack=None),
    color = alt.Color('Grades:N',scale = alt.Scale(domain=domain_1, range=range_1))).properties(width=200)

In [472]:
# DRAW OUT AND LABEL PERCENTAGE DIFFERENCES FOR THESE FINAL GRAPHS - DO THEM FOR TRUMP AND CLINTON

layered_Arizona_phone = alt.Chart(fin_df_Arizona_phone,title = "Arizona").transform_fold(
    ['All Polls (Trump)','Grade B and Above (Trump)'], as_=['Grades','Percentage']).mark_bar(opacity=1).encode(
    x = alt.X('label:N', axis=alt.Axis( title = '')),
    y = alt.Y('Percentage:Q', scale = alt.Scale(domain = [35,50]), stack=None,axis = alt.Axis(title = "Percent Vote (%)", titleFontSize = 13, titleFontWeight = 500)),
    #column = 'label:N',
    color = alt.Color('Grades:N',scale = alt.Scale(domain=domain_1, range=range_1),legend = alt.Legend(legendX = -50))).properties(width=200)

layered_Arizona_online = alt.Chart(fin_df_Arizona_online).transform_fold(
    ['Grade B and Above (Trump)', 'All Polls (Trump)'], as_=['Grades','Percentage']).mark_bar(opacity=1).encode(
    x = alt.X('label:N', axis=alt.Axis( title = '')),
    y = alt.Y('Percentage:Q', scale = alt.Scale(domain = [35,50]), stack=None, axis=alt.Axis(tickSize = 15)),
    #column = 'label:N',
    color = alt.Color('Grades:N',scale = alt.Scale(domain=domain_1, range=range_1),legend = alt.Legend(legendX = -50))).properties(width=200)


layered_Florida_phone = alt.Chart(fin_df_Florida_phone, title = "Florida").transform_fold(
    ['All Polls (Trump)','Grade B and Above (Trump)'], as_=['Grades','Percentage']).mark_bar(opacity=1).encode(
    x = alt.X('label:N', axis=alt.Axis( title = '')),
    y = alt.Y('Percentage:Q', scale = alt.Scale(domain = [35,50]), stack=None),
    #column = 'label:N',
    color = alt.Color('Grades:N',scale = alt.Scale(domain=domain_1, range=range_1),legend = alt.Legend(legendX = -50))).properties(width=200)

layered_Florida_online = alt.Chart(fin_df_Florida_online).transform_fold(
    ['All Polls (Trump)', 'Grade B and Above (Trump)'], as_=['Grades','Percentage']).mark_bar(opacity=1).encode(
    x = alt.X('label:N', axis=alt.Axis( title = '')),
    y = alt.Y('Percentage:Q', scale = alt.Scale(domain = [35,50]), stack=None, axis=alt.Axis(labels = False, title = '', ticks = False)),
    #column = 'label:N',
    color = alt.Color('Grades:N',scale = alt.Scale(domain=domain_1, range=range_1),legend = alt.Legend(legendX = -50))).properties(width=200)

################################

layered_North_Carolina_phone = alt.Chart(fin_df_North_Carolina_phone, title = "North Carolina").transform_fold(
    ['All Polls (Trump)','Grade B and Above (Trump)'], as_=['Grades','Percentage']).mark_bar(opacity=1).encode(
    x = alt.X('label:N', axis=alt.Axis( title = '')),
    y = alt.Y('Percentage:Q', scale = alt.Scale(domain = [35,50]), stack=None),
    #column = 'label:N',
    color = alt.Color('Grades:N',scale = alt.Scale(domain=domain_1, range=range_1),legend = alt.Legend(legendX = -50))
).properties(width=200)

layered_North_Carolina_online = alt.Chart(fin_df_North_Carolina_online).transform_fold(
    ['Grade B and Above (Trump)', 'All Polls (Trump)'], as_=['Grades','Percentage']).mark_bar(opacity=1).encode(
    x = alt.X('label:N', axis=alt.Axis( title = '')),
    y = alt.Y('Percentage:Q', scale = alt.Scale(domain = [35,50]), stack=None, axis=alt.Axis(labels = False, title = '', ticks = False)),
    #column = 'label:N',
    color = alt.Color('Grades:N',scale = alt.Scale(domain=domain_1, range=range_1),legend = alt.Legend(legendX = -50))).properties(width=200)


layered_Ohio_phone = alt.Chart(fin_df_Ohio_phone,title = "Ohio").transform_fold(
    ['All Polls (Trump)','Grade B and Above (Trump)'], as_=['Grades','Percentage']).mark_bar(opacity=1).encode(
    x = alt.X('label:N', axis=alt.Axis( title = '')),
    y = alt.Y('Percentage:Q', scale = alt.Scale(domain = [35,50]), stack=None),
    #column = 'label:N',
    color = alt.Color('Grades:N',scale = alt.Scale(domain=domain_1, range=range_1),legend = alt.Legend(legendX = -50))).properties(width=200)

layered_Ohio_online = alt.Chart(fin_df_Ohio_online).transform_fold(
    ['Grade B and Above (Trump)', 'All Polls (Trump)'], as_=['Grades','Percentage']).mark_bar(opacity=1).encode(
    x = alt.X('label:N', axis=alt.Axis( title = '')),
    y = alt.Y('Percentage:Q', scale = alt.Scale(domain = [35,50]), stack=None, axis=alt.Axis(labels = False, title = '', ticks = False)),
    #column = 'label:N',
    color = alt.Color('Grades:N',scale = alt.Scale(domain=domain_1, range=range_1),legend = alt.Legend(legendX = -50))).properties(width=200)


layered_Pennsylvania_phone = alt.Chart(fin_df_Pennsylvania_phone,title = "Pennsylvania").transform_fold(
    ['All Polls (Trump)','Grade B and Above (Trump)'], as_=['Grades','Percentage']).mark_bar(opacity=1).encode(
    x = alt.X('label:N', axis=alt.Axis( title = '')),
    y = alt.Y('Percentage:Q', scale = alt.Scale(domain = [35,50]), stack=None),
    #column = 'label:N',
    color = alt.Color('Grades:N',scale = alt.Scale(domain=domain_1, range=range_1),legend = alt.Legend(legendX = -50))).properties(width=200)

layered_Pennsylvania_online = alt.Chart(fin_df_Pennsylvania_online).transform_fold(
    ['All Polls (Trump)', 'Grade B and Above (Trump)'], as_=['Grades','Percentage']).mark_bar(opacity=1).encode(
    x = alt.X('label:N', axis=alt.Axis( title = '')),
    y = alt.Y('Percentage:Q', scale = alt.Scale(domain = [35,50]), stack=None, axis=alt.Axis(labels = False, title = '', ticks = False)),
    #column = 'label:N',
    color = alt.Color('Grades:N',scale = alt.Scale(domain=domain_1, range=range_1), 
                      legend = alt.Legend(orient = 'bottom', labelFontSize = 12, titleFontSize = 13), title = "Pollster Grades")).properties(width=200)


(layered_Arizona_phone + layered_Arizona_online | layered_Florida_phone + 
 layered_Florida_online | layered_North_Carolina_phone + layered_North_Carolina_online | 
 layered_Ohio_phone + layered_Ohio_online | 
 layered_Pennsylvania_phone + layered_Pennsylvania_online).properties(
    title = alt.TitleParams(text = 'Change in Poll Percentage from All Polls to Highly Rate Polls',
                           subtitle = 'Donald Trump'))


alt.HConcatChart(...)

In [473]:
# Clinton graphs new

domain_2 = ['Grade B and Above (Clinton)', 'All Polls (Clinton)']
range_2 = ['#2d3142','#274c77']

layered_Arizona_phone = alt.Chart(fin_df_Arizona_phone,title = "Arizona").transform_fold(
    ['All Polls (Clinton)','Grade B and Above (Clinton)'], as_=['Grades','Percentage']).mark_bar(opacity=1).encode(
    x = alt.X('label:N', axis=alt.Axis( title = '')),
    y = alt.Y('Percentage:Q', scale = alt.Scale(domain = [35,50]), axis = alt.Axis(title = "Percent Vote (%)", titleFontSize = 13, titleFontWeight = 500), stack=None),
    #column = 'label:N',
    color = alt.Color('Grades:N',scale = alt.Scale(domain=domain_2, range=range_2))).properties(width=200)

layered_Arizona_online = alt.Chart(fin_df_Arizona_online).transform_fold(
    ['All Polls (Clinton)', 'Grade B and Above (Clinton)'], as_=['Grades','Percentage']).mark_bar(opacity=1).encode(
    x = alt.X('label:N', axis=alt.Axis( title = '')),
    y = alt.Y('Percentage:Q', scale = alt.Scale(domain = [35,50]), stack=None),
    #column = 'label:N',
    color = alt.Color('Grades:N',scale = alt.Scale(domain=domain_2, range=range_2))).properties(width=200)


layered_Florida_phone = alt.Chart(fin_df_Florida_phone, title = "Florida").transform_fold(
    ['All Polls (Clinton)','Grade B and Above (Clinton)'], as_=['Grades','Percentage']).mark_bar(opacity=1).encode(
    x = alt.X('label:N', axis=alt.Axis( title = '')),
    y = alt.Y('Percentage:Q', scale = alt.Scale(domain = [35,50]), stack=None),
    #column = 'label:N',
    color = alt.Color('Grades:N',scale = alt.Scale(domain=domain_2, range=range_2))).properties(width=200)

layered_Florida_online = alt.Chart(fin_df_Florida_online).transform_fold(
    ['All Polls (Clinton)', 'Grade B and Above (Clinton)'], as_=['Grades','Percentage']).mark_bar(opacity=1).encode(
    x = alt.X('label:N', axis=alt.Axis( title = '')),
    y = alt.Y('Percentage:Q', scale = alt.Scale(domain = [35,50]), stack=None, axis=alt.Axis(labels = False, title = '')),
    #column = 'label:N',
    color = alt.Color('Grades:N',scale = alt.Scale(domain=domain_2, range=range_2))).properties(width=200)

################################

layered_North_Carolina_phone = alt.Chart(fin_df_North_Carolina_phone, title = "North Carolina").transform_fold(
    ['All Polls (Clinton)','Grade B and Above (Clinton)'], as_=['Grades','Percentage']).mark_bar(opacity=1).encode(
    x = alt.X('label:N', axis=alt.Axis( title = '')),
    y = alt.Y('Percentage:Q', scale = alt.Scale(domain = [35,50]), stack=None),
    #column = 'label:N',
    color = alt.Color('Grades:N',scale = alt.Scale(domain=domain_2, range=range_2))).properties(width=200)

layered_North_Carolina_online = alt.Chart(fin_df_North_Carolina_online).transform_fold(
    ['Grade B and Above (Clinton)', 'All Polls (Clinton)'], as_=['Grades','Percentage']).mark_bar(opacity=1).encode(
    x = alt.X('label:N', axis=alt.Axis( title = '')),
    y = alt.Y('Percentage:Q', scale = alt.Scale(domain = [35,50]), stack=None, axis=alt.Axis(labels = False, title = '')),
    #column = 'label:N',
    color = alt.Color('Grades:N',scale = alt.Scale(domain=domain_2, range=range_2))).properties(width=200)


layered_Ohio_phone = alt.Chart(fin_df_Ohio_phone,title = "Ohio").transform_fold(
    ['All Polls (Clinton)','Grade B and Above (Clinton)'], as_=['Grades','Percentage']).mark_bar(opacity=1).encode(
    x = alt.X('label:N', axis=alt.Axis( title = '')),
    y = alt.Y('Percentage:Q', scale = alt.Scale(domain = [35,50]), stack=None),
    #column = 'label:N',
    color = alt.Color('Grades:N',scale = alt.Scale(domain=domain_2, range=range_2))).properties(width=200)

layered_Ohio_online = alt.Chart(fin_df_Ohio_online).transform_fold(
    ['Grade B and Above (Clinton)', 'All Polls (Clinton)'], as_=['Grades','Percentage']).mark_bar(opacity=1).encode(
    x = alt.X('label:N', axis=alt.Axis( title = '')),
    y = alt.Y('Percentage:Q', scale = alt.Scale(domain = [35,50]), stack=None, axis=alt.Axis(labels = False, title = '')),
    #column = 'label:N',
    color = alt.Color('Grades:N',scale = alt.Scale(domain=domain_2, range=range_2))).properties(width=200)


layered_Pennsylvania_phone = alt.Chart(fin_df_Pennsylvania_phone,title = "Pennsylvania").transform_fold(
    ['All Polls (Clinton)','Grade B and Above (Clinton)'], as_=['Grades','Percentage']).mark_bar(opacity=1).encode(
    x = alt.X('label:N', axis=alt.Axis( title = '')),
    y = alt.Y('Percentage:Q', scale = alt.Scale(domain = [35,50]), stack=None),
    #column = 'label:N',
    color = alt.Color('Grades:N',scale = alt.Scale(domain=domain_2, range=range_2))).properties(width=200)

layered_Pennsylvania_online = alt.Chart(fin_df_Pennsylvania_online).transform_fold(
    ['All Polls (Clinton)', 'Grade B and Above (Clinton)'], as_=['Grades','Percentage']).mark_bar(opacity=1).encode(
    x = alt.X('label:N', axis=alt.Axis( title = '')),
    y = alt.Y('Percentage:Q', scale = alt.Scale(domain = [35,50]), stack=None, axis=alt.Axis(labels = False, title = '')),
    #column = 'label:N',
    color = alt.Color('Grades:N',scale = alt.Scale(domain=domain_2, range=range_2),
                     legend = alt.Legend(orient = 'bottom', labelFontSize = 12, titleFontSize = 13), title = "Pollster Grades")).properties(width=200)


(layered_Arizona_phone + layered_Arizona_online| layered_Florida_phone + layered_Florida_online | 
layered_North_Carolina_phone + layered_North_Carolina_online | layered_Ohio_phone + layered_Ohio_online | 
layered_Pennsylvania_phone + layered_Pennsylvania_online).properties(
    title = alt.TitleParams(text = 'Change in Poll Percentage from All Polls to Highly Rate Polls',
                           subtitle = 'Hilary Clinton'))

# make combined charts with same title for each state than use | to concatenate them horizantally

alt.HConcatChart(...)

# Clinton graphs - old

# Without Colorado and New Hampshire
## MAKE ONE FOR EACH STATE / U.S.

domain_2 = ['clinton_good', 'clinton_all']
range_2 = ['#2d3142','#274c77'] # make clinton the opposite? red sticking out blue on bottom? (so looks like blue fill)

layered_Arizona = alt.Chart(fin_df_Arizona).transform_fold(
    ['clinton_good','clinton_all'], as_=['Grades','Percentage']).mark_bar(opacity=1).encode(
    x = alt.X('label:N'),
    y = alt.Y('Percentage:Q', scale = alt.Scale(domain = [35,52]), stack=None),
    color = alt.Color('Grades:N',scale = alt.Scale(domain=domain_2, range=range_2))).properties(width=200)


layered_Florida = alt.Chart(fin_df_Florida).transform_fold(
    ['clinton_all','clinton_good'], as_=['Grades','Percentage']).mark_bar(opacity=1).encode(
    x = alt.X('label:N'),
    y = alt.Y('Percentage:Q', scale = alt.Scale(domain = [35,52]), stack=None),
    color = alt.Color('Grades:N',scale = alt.Scale(domain=domain_2, range=range_2))).properties(width=200)


layered_North_Carolina = alt.Chart(fin_df_North_Carolina).transform_fold(
    ['clinton_all','clinton_good'], as_=['Grades','Percentage']).mark_bar(opacity=1).encode(
    x = alt.X('label:N'),
    y = alt.Y('Percentage:Q', scale = alt.Scale(domain = [35,52]), stack=None),
    color = alt.Color('Grades:N',scale = alt.Scale(domain=domain_2, range=range_2))).properties(width=200)


layered_Ohio = alt.Chart(fin_df_Ohio).transform_fold(
    ['clinton_all','clinton_good'], as_=['Grades','Percentage']).mark_bar(opacity=1).encode(
    x = alt.X('label:N'),
    y = alt.Y('Percentage:Q', scale = alt.Scale(domain = [35,52]), stack=None),
    color = alt.Color('Grades:N',scale = alt.Scale(domain=domain_2, range=range_2))).properties(width=200)


layered_Pennsylvania = alt.Chart(fin_df_Pennsylvania).transform_fold(
    ['clinton_all','clinton_good'], as_=['Grades','Percentage']).mark_bar(opacity=1).encode(
    x = alt.X('label:N'),
    y = alt.Y('Percentage:Q', scale = alt.Scale(domain = [35,52]), stack=None),
    color = alt.Color('Grades:N',scale = alt.Scale(domain=domain_2, range=range_2))).properties(width=200)


layered_Florida | layered_Arizona | layered_North_Carolina | layered_Ohio | layered_Pennsylvania

In [295]:
domain_4 = ['Election Vote (Trump)', 'Online Polls (Trump)', 'Phone Polls (Trump)']
range_4 = ['#9a031e',  '#b0bcd1', '#9e9186']

domain_5 = ['Election Vote (Clinton)','Online Polls (Clinton)','Phone Polls (Clinton)']

range_5 = ['#274c77','#b0bcd1','#9e9186']

In [448]:
final_graph_trump_all = alt.Chart(fin_df_all).transform_fold(
    ['Online Polls (Trump)','Election Vote (Trump)','Phone Polls (Trump)']).mark_bar().encode(
    column = alt.Column('state', header = alt.Header(labelFontSize=13), title = None),
    x = alt.X('key:N', title = None, axis = alt.Axis(ticks = False, labels = False)),
    y = alt.Y('value:Q', scale = alt.Scale(domain = [35,54]), axis = alt.Axis(title = "Percent Vote (%)", titleFontSize = 13, titleFontWeight = 500)),
    color = alt.Color('key:N', scale = alt.Scale(domain = domain_4, range = range_4),legend = alt.Legend(labelFontSize = 11, orient = 'bottom'), title = None)
).properties(title = alt.TitleParams(text = 'Polling vs Election Vote',
                           subtitle = 'Donald Trump'), width = 75).configure_legend(labelFontSize = 11)#.configure_facet(spacing = 5)

final_graph_trump_all

alt.Chart(...)

final_graph_trump_good = alt.Chart(fin_df_good).transform_fold(
    ['trump_phone_pct','trump_online_pct','trump_elect_pct']).mark_bar().encode(
    column = alt.Column('state'),
    x = alt.X('key:N'),
    y = alt.Y('value:Q', scale = alt.Scale(domain = [35,52])),
    color = alt.Color('key:N')
)#.configure_facet(spacing = 5)

final_graph_trump_good

In [297]:
domain_6 = ['Online','Phone']

range_6 = ['#b0bcd1','#9e9186']

In [363]:
# Graph of number of phone polls in each, and number of online polls in each

count_df = pd.DataFrame({'Poll Grades':['All Grades', 'Grade B and Above'],
                        'Online': [online_data.shape[0], online_data_good.shape[0]],
                        'Phone': [phone_data.shape[0], phone_data_good.shape[0]]})

count_graph = alt.Chart(count_df).transform_fold(['Phone','Online']).mark_line(point=True, strokeWidth = 3.2).encode(
x= alt.X('Poll Grades:N', title = None, axis = alt.Axis(labelAngle=0)),
y= alt.Y('value:Q', scale= alt.Scale(domain = [15,105]), axis = alt.Axis(title = "Number of Polls", 
                                                                         titleFontSize = 11, titleFontWeight = 500)),
color = alt.Color('key:N', scale = alt.Scale(domain = domain_6, range = range_6), legend = alt.Legend(labelFontSize = 11), 
                  title = None)).properties(width=400,
    height=150)

text = alt.Chart(count_df).transform_fold(['Phone','Online']).mark_text(dy = -10, color='black').encode(
    x=alt.X('Poll Grades:N', stack='zero'),
    y=alt.Y('value:Q'),
    text=alt.Text('value:Q', )).properties(width=400,
    height=150)

count_graph+text

alt.LayerChart(...)

In [299]:
phone_data_good.shape[0]

51

In [300]:
# Number of polls

count_df_all = pd.DataFrame({#'label':['All Pollsters'],
                        'Medium':['Online', 'Phone'],
                        'Number': [online_data.shape[0],phone_data.shape[0]]})


In [301]:
bars = alt.Chart(count_df_all, title = "Number of Phone and Online Polls").mark_bar().encode(
    x=alt.X('Number:Q', stack='zero', axis=alt.Axis(title = '')),
    y=alt.Y('label:N', axis = None),
    color=alt.Color('Medium:N', scale = alt.Scale(domain = domain_6, range = range_6), title = None ,sort = ['Phone', 'Online'])
)

text = alt.Chart(count_df_all).mark_text(dx=-17, dy=3, color='white').encode(
    x=alt.X('Number:Q', stack='zero'),
    y=alt.Y('label:N'),
    detail='Medium:N',
    text=alt.Text('Number:Q', format='i', title = None)
)

(bars + text).configure_title(fontSize=11, anchor = 'start', dy=-7)

alt.LayerChart(...)

In [447]:
# clinton graphs comparing good and all
final_graph_clinton_all = alt.Chart(fin_df_all).transform_fold(
    ['Phone Polls (Clinton)','Online Polls (Clinton)','Election Vote (Clinton)']).mark_bar().encode(
    column = alt.Column('state', header = alt.Header(labelFontSize=13), title = None),
    x = alt.X('key:N', title = None, axis = alt.Axis(ticks = False, labels = False)),
    y = alt.Y('value:Q', scale = alt.Scale(domain = [35,54]), axis = alt.Axis(title = "Percent Vote (%)", titleFontSize = 13, titleFontWeight = 500)),
    color = alt.Color('key:N', scale = alt.Scale(domain = domain_5, range = range_5),legend = alt.Legend(labelFontSize = 11, orient = 'bottom'), title = None)
).properties(title = alt.TitleParams(text = 'Polling vs Election Vote',
                           subtitle = 'Hilary Clinton'),width=75).configure_legend(labelFontSize = 11)#.configure_facet(spacing = 5)

final_graph_clinton_all

alt.Chart(...)

# clinton graphs comparing good and all
final_graph_clinton_good = alt.Chart(fin_df_good).transform_fold(
    ['clinton_phone_pct','clinton_online_pct','clinton_elect_pct']).mark_bar().encode(
    column = alt.Column('state'),
    x = alt.X('key:N'),
    y = alt.Y('value:Q', scale = alt.Scale(domain = [35,52])),
    color = alt.Color('key:N')
)#.configure_facet(spacing = 5)

final_graph_clinton_good

In [303]:
# scaled graphs for Trump and Clinton

final_graph_trump_all_100 = alt.Chart(fin_df_all_100).transform_fold(
    ['trump_phone_pct','trump_online_pct','trump_elect_pct']).mark_bar().encode(
    column = alt.Column('state'),
    x = alt.X('key:N'),
    y = alt.Y('value:Q', scale = alt.Scale(domain = [35,55])),
    color = alt.Color('key:N', scale = alt.Scale(domain = domain_4, range = range_4))
).properties(width = 140)#.configure_facet(spacing = 5)

final_graph_trump_all_100

In [446]:
# clinton graphs comparing good and all
final_graph_clinton_all_100 = alt.Chart(fin_df_all_100).transform_fold(
    ['Phone Polls (Clinton)','Online Polls (Clinton)','Election Vote (Clinton)']).mark_bar().encode(
    column = alt.Column('state', header = alt.Header(labelFontSize=13), title = None),
    x = alt.X('key:N', title = None, axis = alt.Axis(ticks = False, labels = False)),
    y = alt.Y('value:Q', scale = alt.Scale(domain = [35,54]), axis = alt.Axis(title = "Percent Vote (%)", titleFontSize = 13, titleFontWeight = 500)),
    color = alt.Color('key:N', scale = alt.Scale(domain = domain_5, range = range_5), 
                      legend = alt.Legend(labelFontSize = 11, orient = 'bottom'), title = None),
).properties(title = alt.TitleParams(text = 'Polling vs Election Vote - Percent Total Vote of Two Main Candidates',
                           subtitle = 'Hilary Clinton'),width=75)#.configure_facet(spacing = 5)


final_graph_clinton_all_100

alt.Chart(...)

In [53]:
# Definitive Graphics - layered bar charts, on for trump, one for clinton

#final_graph_trump_layered = alt.Chart(fin_df_both).transform_fold()

In [101]:
# state - number of electoral votes - winner

elec_df = pd.DataFrame({'state': ['AZ', 'CO', 'FL', 'IA', 'MI', 'NV', 'NH',
                                  'NC','OH','PA','VA','WI'],
                      'won_by_pop':['Trump', 'Clinton', 'Trump','Trump','Trump','Clinton',
                                    'Clinton', 'Trump','Trump', 'Trump','Clinton', 'Trump'],
                      'num_seats_winner': [11,9,29,6,16,6,4,15,18,20,13,10]})

domain_3 = ['Trump','Clinton']
range_3 = ['#9a031e','#274c77']


In [431]:
bars = alt.Chart(elec_df).mark_bar().encode(
    x=alt.X('state:N', stack='zero', title = None),
    y=alt.Y('num_seats_winner:Q', ),
    color=alt.Color('won_by_pop:N', scale = alt.Scale(domain = domain_3, range=range_3),legend = alt.Legend(orient = 'bottom'), title = 'Winner')
).properties(width=500, height = 200)

# Removed text config  .mark_text(angle = 270, baseline = 'middle', fontWeight = 700, fontSize = 15)

text = alt.Chart(elec_df).mark_text(dy = -5).encode(
    x=alt.X('state:N'),
    y=alt.Y('num_seats_winner:Q', scale = alt.Scale(domain = [0,35]), axis=None),
    text=alt.Text('num_seats_winner:Q')
).properties(width=500)

(bars + text).properties(
        title = {"text": "Electoral Votes",
                 "subtitle": "(Battleground States)"            
        }).configure_axis(labelFontSize=13, labelAngle = 0, labelFontWeight = 400
                            ).configure_title(fontSize=15, anchor = 'start', dy=0)


alt.LayerChart(...)

final_graph_clinton = alt.Chart(fin_df_all).transform_fold(
    ['clinton_phone_pct','clinton_online_pct','clinton_elect_pct']).mark_bar().encode(
    column = alt.Column('state'),
    x = alt.X('key:N'),
    y = alt.Y('value:Q', scale = alt.Scale(domain = [35,52])),
    color = alt.Color('key:N')
)#.configure_facet(spacing = 5)

final_graph_clinton

# These next two are MIDWAY GRAPHS - not final

final_graph_phone = alt.Chart(fin_df).transform_fold(
    ['trump_phone_pct','clinton_phone_pct']).mark_bar().encode(
    column = alt.Column('state'),
    x = alt.X('key:N'),
    y = alt.Y('value:Q'),
    color = alt.Color('key:N')
)#.configure_facet(spacing = 5)

final_graph_phone

In [56]:
final_graph_online = alt.Chart(fin_df).transform_fold(
    ['trump_online_pct','clinton_online_pct']).mark_bar().encode(
    column = alt.Column('state'),
    x = alt.X('key:N'),
    y = alt.Y('value:Q'),
    color = alt.Color('key:N')
)#.configure_facet(spacing = 5)

final_graph_online

NameError: name 'fin_df' is not defined

# Look at difference between percentage for phone and online

fin_df['online-phone_trump'] = fin_df['trump_online_pct'] - fin_df['trump_phone_pct']
fin_df['online-phone_clinton'] = fin_df['clinton_online_pct'] - fin_df['clinton_phone_pct']
fin_df

# Look at difference between percentage for phone and online
# stretch both to 100% first to get comparable measures

fin_df['online-phone_trump'] = 100 - ((100/fin_df['trump_online_pct'])*fin_df['trump_phone_pct'])
fin_df['online-phone_clinton'] = 100 - ((100/fin_df['clinton_online_pct'])*fin_df['clinton_phone_pct'])
fin_df